The FWS Threatened and Endangered Species (TESS) system is part of their underlying data source. TESS has a set of web services that are a little rudimentary but functional. In this notebook, we retrieve information from TESS, first for all of the species we "ECOS-scraped" that have ITIS TSN numbers declared (cached to the cache/workplan_species.json file), and then we look for any other species in the list where we did not get a TSN.

The main bispy function here is the search function in the Tess class of the tess module. It takes either an ITIS TSN or scientific name and runs a search. The TESS service returns XML, but the function transforms and tweaks slightly to return a dictionary (JSON) structure.

In [1]:
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed
import random

tess = bispy.tess.Tess()

In [2]:
# Open up the cached workplan species
with open("cache/workplan_species.json", "r") as f:
    workplan_species = json.loads(f.read())

In [3]:
# Prepare two lists - one of the TSNs we already know about from ECOS scraping and the other of the remaining scientific names
tsn_list = [r["ITIS TSN"] for r in workplan_species if r["ITIS TSN"] is not None]
names_without_tsns = [r["Scientific Name"] for r in workplan_species if r["ITIS TSN"] is None]

In [4]:
# Use joblib to run multiple requests for TESS documents in parallel via known ITIS TSNs
tess_cache_from_tsn = Parallel(n_jobs=8)(delayed(tess.search)(tsn) for tsn in tsn_list)

In [5]:
# Use joblib to run multiple requests for TESS documents in parallel via scientific names
tess_cache_from_names = Parallel(n_jobs=8)(delayed(tess.search)(name) for name in names_without_tsns)

In [6]:
# If any new TSNs were found via name search, update those back to the workplan species JSON
updated_workplan_species = list()
for record in [t for t in tess_cache_from_names if 'TESS Species' in t.keys() and int(t['TESS Species']['SPECIES_DETAIL']['TSN']) > 0]:
    workplan_record = next((d for d in workplan_species if d["Scientific Name"] == record['TESS Species']['SPECIES_DETAIL']['SCINAME']), None)
    workplan_record["ITIS TSN"] = record["TESS Species"]["SPECIES_DETAIL"]["TSN"]
    updated_workplan_species.append(workplan_record)
    updated_species_names = [s["Scientific Name"] for s in updated_workplan_species]

if len(updated_workplan_species) > 0:
    new_workplan_species = list(filter(lambda i: i['Scientific Name'] not in updated_species_names, workplan_species))
    new_workplan_species.extend(updated_workplan_species)
    with open("cache/workplan_species.json", "w") as f:
        f.write(json.dumps(new_workplan_species, indent=4))
    

In [7]:
# Combine and cache the data from TESS
tess_cache_from_tsn.extend(tess_cache_from_names)
with open("cache/tess_data.json", "w") as f:
    f.write(json.dumps(tess_cache_from_tsn, indent=4))